# Pytest

[Pytest](https://docs.pytest.org/en/latest/contents.html) has rapidly established itself as the standard python testing framework. Its power lies in its simplicity, which makes it super easy to write tests and to run them. Tests can be as simple as the ones that we've already seen, so it is perfect for small projects. However, hidden behind the simple exterior, `pytest` provides a great deal of power and flexibility, meaning that it scales well to large and complex projects.

In this section we will learn how to run tests using `pytest` and how to take advantage of the powerful `pytest.mark` helper in order to add useful attributes to our tests.

## Running tests

Pytest comes with a command-line tool called, unsurprisingly, `pytest`. (With older versions of python this was called `py.test`.) When run, `pytest` searches for test files in all directories and files below the current directory, collects the tests together, then runs them. Pytest uses name matching to locate the test files. Valid names start or end with `test`, e.g.

```bash
test_example.py    # We will use this as our standard naming convention.
example_test.py
```

(Note that this naming convention applies to test functions, as well as files.)

You can specify one or more paths and `pytest` will only look for test files in those paths, e.g.

```bash
pytest /path/to/my/awesome/module
```

When writing a python module it is good practice to set up a directory structure in order to keep things tidy. Throughout this course we will use the following:

```bash
mypkg/
    __init__.py
    whizz.py
    bang.py
    test/
        __init__.py
        test_whizz.py
        test_bang.py
```

Here the `__init__.py` files makes python aware that the directories should be treated as modules. Assuming we're in the top level directory (where out notebooks reside) this allows us to do the following:

```python
import mypkg
```

Let's dive in and run some tests using `mymodule` that was introduced in the previous section.

In [22]:
! pytest mymodule

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 16 items                                                             

mymodule/test/test_errors.py .s                                          [ 12%]
mymodule/test/test_mymath.py F.......s..Fx.                              [100%]

=================================== FAILURES ===================================
___________________________________ test_add ___________________________________

    def test_add():
        """ Test the add function. """
    
        assert add(1, 1) ==  2
>       assert add(1, 2) == add(2, 1) == 3
E       assert 2 == 4
E        +  where 2 = add(1, 2)
E        +  and   4 = add(2, 1)

mymodule/test/test_mymath.py:11: AssertionError
____________________________ test_greaterThan[3-7] _____________________________
[XP

What just happened?

Well, `pytest` searched within the `mymodule` directory and collected a total of 16 tests. These tests were spread accross two files:

```bash
mymodule/test/test_errors.py
mymodule/test/test_mymath.py
```

The tests were then run with a single failure reported for the test `test_add`. This was a test for the `add` function, which we found to be broken in the previous section.

What are all the cryptic symbols next to the name of the each files?

```bash
F.......s..Xx
```

To get more detailed information about each test, we can run `pytest` in _verbose_ mode.

In [34]:
! pytest mymodule -v

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0 -- /usr/bin/python
cachedir: .cache
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 16 items                                                             

mymodule/test/test_errors.py::test_indexError PASSED                     [  6%]
mymodule/test/test_errors.py::test_BSoD SKIPPED                          [ 12%]
mymodule/test/test_mymath.py::test_add FAILED                            [ 18%]
mymodule/test/test_mymath.py::test_sub[1-2--1] PASSED                    [ 25%]
mymodule/test/test_mymath.py::test_sub[7-3-4] PASSED                     [ 31%]
mymodule/test/test_mymath.py::test_sub[21-58--37] PASSED                 [ 37%]
mymodule/test/test_mymath.py::test_mul[1-3] PASSED                       [ 43%]
mymodule/test/test_mymath.py::test_mul[1-4] PASSED                       [ 50%]
mymod

We now have more detailed information about each test. Matching up the output with the symbols we can see that...

```
. = PASSED
s = SKIPPED
F = FAILED
x = xfail
X = XPASS
```

What do `xfail` and `XPASS` mean, and why were some tests skipped? Also, note that some tests were run mutliple times, e.g. `test_sub`. What do the numbers in square brackets mean?

To _report_ more information on tests that were `SKIPPED`, `XPASS`, or `xfail`, we can run `pytest` as follows.

In [21]:
! pytest mymodule -rsxX

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 16 items                                                             

mymodule/test/test_errors.py .s                                          [ 12%]
mymodule/test/test_mymath.py F.......s..Xx.                              [100%]
=========================== short test summary info ============================
SKIP [1] mymodule/test/test_errors.py:12: Only runs on windows.
SKIP [1] mymodule/test/test_mymath.py:29: Not yet implemented.
XFAIL mymodule/test/test_mymath.py::test_greaterThan[6-11]
XPASS mymodule/test/test_mymath.py::test_greaterThan[3-7] 

=================================== FAILURES ===================================
___________________________________ test_add ___________________________________

    def test_add():
        """

We now have addition information about these tests. One test was skipped because it is only valid on Windows, another because it was testing functionality that hasn't been implemented yet.

By now, you might have guessed that `xfail` means _expected to fail_. You can see that one test, `test_greaterThan`, had and expected failure. Here the numbers in square brackets indicate the arguments to the test function. We are expecting that 6 isn't greater than 11, which is indeed correctly reported as an expected failure.

But hang on, what about the `XPASS`? Is this an _expected pass_?

No, `XPASS` means that an expected _failure_ actually _passed_! We were testing that 3 shouldn't be greater than 7, but somehow it passed. This means that there must be a bug in our code. (Note, however, that `test_greaterThan` didn't appear in the list of failures that were output by `pytest`. We'll cover this in the next section.)

Phew, if all that was too much then you can always run `pytest` in _quiet_ mode. Here we only see a minimal output showing the progress of the tests and reports for any failures.

In [23]:
! pytest mymodule -q

.sF.......s..Xx.                                                         [100%]
=================================== FAILURES ===================================
___________________________________ test_add ___________________________________

    def test_add():
        """ Test the add function. """
    
        assert add(1, 1) ==  2
>       assert add(1, 2) == add(2, 1) == 3
E       assert 2 == 4
E        +  where 2 = add(1, 2)
E        +  and   4 = add(2, 1)

mymodule/test/test_mymath.py:11: AssertionError
1 failed, 11 passed, 2 skipped, 1 xfailed, 1 xpassed in 8.71 seconds


# Exercises

More details on how to invoke `pytest` can be found [here](https://docs.pytest.org/en/latest/usage.html).

__Exercise 1:__ Make `pytest` stop after the first failure.

In [24]:
! pytest mymodule -x

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 16 items                                                             

mymodule/test/test_errors.py .s                                          [ 12%]
mymodule/test/test_mymath.py F

=================================== FAILURES ===================================
___________________________________ test_add ___________________________________

    def test_add():
        """ Test the add function. """
    
        assert add(1, 1) ==  2
>       assert add(1, 2) == add(2, 1) == 3
E       assert 2 == 4
E        +  where 2 = add(1, 2)
E        +  and   4 = add(2, 1)

mymodule/test/test_mymath.py:11: AssertionError
================ 1 failed, 1 passed, 1 skipped in 0.04 seconds =================


__Exercise 2:__ Only run tests for the `test_mul` function.

In [32]:
! pytest mymodule/test/test_mymath.py::test_mul

============================= test session starts ==============================
platform linux -- Python 3.6.3, pytest-3.3.0, py-1.5.0, pluggy-0.6.0
rootdir: /home/lester/Code/siremol.org/chryswoods.com/python_and_data/testing, inifile:
collected 4 items                                                              

mymodule/test/test_mymath.py ....                                        [100%]

=========================== 4 passed in 0.01 seconds ===========================
